<a href="https://colab.research.google.com/github/doublelockcup/Pathology-Images-Classification/blob/master/HW5_Tianbo_Qiu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow-gpu==2.0.0-alpha0
!pip install sacrebleu # https://github.com/mjpost/sacreBLEU

    100% |████████████████████████████████| 332.1MB 48kB/s 
    100% |████████████████████████████████| 3.0MB 8.7MB/s 
    100% |████████████████████████████████| 61kB 25.6MB/s 
    100% |████████████████████████████████| 419kB 4.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/56/c0/fb/1c7f9b3a71f64cdf86291cc645596f71746807bf2f72b3c1dd
Successfully built sacrebleu


In [2]:
import numpy as np
import re
import sacrebleu
import tensorflow as tf
import time
import unicodedata
print(tf.__version__)

2.0.0-alpha0


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!cp '/content/gdrive/My Drive/DeepLearning/datasets/spa.txt' 'spa.txt'

#Part 1

Translate English to Spanish

In [0]:
spa = open('spa.txt', 'r')

In [6]:
data = spa.read().splitlines()
sentences_data = [tuple(d.split('\t')) for d in data]

print(len(sentences_data))
print(sentences_data[:5])

118964
[('Go.', 'Ve.'), ('Go.', 'Vete.'), ('Go.', 'Vaya.'), ('Go.', 'Váyase.'), ('Hi.', 'Hola.')]


In [9]:
def preprocess(s):
  # for details, see https://www.tensorflow.org/alpha/tutorials/sequences/nmt_with_attention
  s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
  s = re.sub(r"([?.!,¿])", r" \1 ", s)
  s = re.sub(r'[" "]+', " ", s)
  s = re.sub(r"[^a-zA-Z?.!,¿]+", " ", s)
  s = s.strip()
  s = '<start> ' + s + ' <end>'
  return s

# sample 3000 sentences as training set
size = 3000
sentences = [(preprocess(source), preprocess(target)) for (source, target) in sentences_data[:size]]
print('Original:', sentences_data[1500])
print('Preprocessed:', sentences[1500])

source_sentences, target_sentences = list(zip(*sentences))

Original: ('We must go.', 'Nos debemos ir.')
Preprocessed: ('<start> We must go . <end>', '<start> Nos debemos ir . <end>')


In [0]:
class Translator:
  def __init__(self, source_sentences, target_sentences):
    self.source_sentences = source_sentences
    self.target_sentences = target_sentences
    self.source_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    self.source_tokenizer.fit_on_texts(source_sentences)
    source_data = self.source_tokenizer.texts_to_sequences(source_sentences)
    self.source_data = tf.keras.preprocessing.sequence.pad_sequences(source_data, padding='post')
    
    self.target_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    self.target_tokenizer.fit_on_texts(target_sentences)
    target_data = self.target_tokenizer.texts_to_sequences(target_sentences)
    self.target_data = tf.keras.preprocessing.sequence.pad_sequences(target_data, padding='post')
    
    self.target_labels = np.zeros(self.target_data.shape)
    self.target_labels[:, 0:self.target_data.shape[1]-1] = self.target_data[:,1:]
    
    self.source_vocab_size = len(self.source_tokenizer.word_index) + 1
    self.target_vocab_size = len(self.target_tokenizer.word_index) + 1
    self.batch_size = batch_size= 10
    self.dataset = tf.data.Dataset.from_tensor_slices((self.source_data, self.target_data, self.target_labels)).batch(batch_size)
    
    

  def decode(self, encoded_data):
    for number in encoded_data:
      if number != 0:
        print("%d -> %s" % (number, self.source_tokenizer.index_word[number]))
        
class Encoder(tf.keras.Model):
  def __init__(self, translator):
    super(Encoder, self).__init__()
    self.embedding_size = 32
    self.rnn_size = 64
    
    self.embedding = tf.keras.layers.Embedding(translator.source_vocab_size, self.embedding_size)
    self.gru = tf.keras.layers.GRU(self.rnn_size, return_sequences=True, return_state=True)
  
  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state=hidden)
    return output, state
  
  def init_state(self, batch_size):
    return tf.zeros((batch_size, self.rnn_size))

class Decoder(tf.keras.Model):
  def __init__(self, translator):
    super(Decoder, self).__init__()
    self.embedding_size = 32
    self.rnn_size = 64
    self.embedding = tf.keras.layers.Embedding(translator.target_vocab_size, self.embedding_size)
    self.gru = tf.keras.layers.GRU(self.rnn_size, return_sequences=True, return_state=True)
    self.dense = tf.keras.layers.Dense(translator.target_vocab_size)
    
  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state=hidden)
    logits = self.dense(output)
    return logits, state
  

def calc_loss(targets, logits):
  mask = tf.math.logical_not(tf.math.equal(targets, 0))
  mask = tf.cast(mask, dtype=tf.int64)
  crossentropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  return crossentropy(targets, logits, sample_weight=mask)



class Trans:
  def __init__(self, source_sentences, target_sentences):
    self.t = t = Translator(source_sentences, target_sentences)
    self.encoder = Encoder(t)
    self.decoder = Decoder(t)
    self.optimizer = tf.keras.optimizers.Adam()
    self.batch_size= batch_size = t.batch_size
   
  def translate_s(self, s):
    encoder = self.encoder
    decoder = self.decoder
    #source_sentences = self.t.source_sentences
    #target_sentences = self.t.target_sentences
    source_data = self.t.source_data
    #target_data = self.t.target_data
    #target_labels = self.t.target_labels
    source_tokenizer = self.t.source_tokenizer
    target_tokenizer = self.t.target_tokenizer
    pad_dim = source_data.shape[1]
    input_sent = source_tokenizer.texts_to_sequences([s])
    input_sent = tf.keras.preprocessing.sequence.pad_sequences(input_sent, maxlen=pad_dim, padding='post')
    #return input_sent
    #input_sent = tf.expand_dims(input_sent, axis=0)
    hidden_state = encoder.init_state(batch_size=1)
    output, hidden_state = encoder(input_sent, hidden_state)
    #return input_sent
    decoder_input = tf.expand_dims([target_tokenizer.word_index['<start>']], 0)
    out_words = []

    decoder_state = hidden_state

    while True:

        decoder_output, decoder_state = decoder(decoder_input, decoder_state)
        decoder_input = tf.argmax(decoder_output, -1)
        word_idx = decoder_input.numpy()[0][0]
        # if we've predicted 0 (which is reserved, usually this will only happen
        # before the decoder is trained, just stop translating and return
        # what we have)
        if word_idx == 0: 
          out_words.append('<end>')
        else:
          out_words.append(target_tokenizer.index_word[word_idx])

        if out_words[-1] == '<end>' or len(out_words) >= 20:
          break

    translation = ' '.join(out_words)    
    return '<start>' + translation
    

    
  
    
    
    
  def translate(self, idx=None):
    encoder = self.encoder
    decoder = self.decoder
    source_sentences = self.t.source_sentences
    target_sentences = self.t.target_sentences
    source_data = self.t.source_data
    target_data = self.t.target_data
    target_labels = self.t.target_labels
    target_tokenizer = self.t.target_tokenizer
    #sentences = list(zip(source_sentences, target_sentences))
    if idx == None: 
      idx = np.random.choice(len(source_sentences))

    input_sent = source_data[idx]
    input_sent = tf.expand_dims(input_sent, axis=0)
    #print('debug---->', input_sent.shape)

    hidden_state = encoder.init_state(batch_size=1)
    output, hidden_state = encoder(input_sent, hidden_state)

    decoder_input = tf.expand_dims([target_tokenizer.word_index['<start>']], 0)
    out_words = []

    decoder_state = hidden_state

    while True:

        decoder_output, decoder_state = decoder(decoder_input, decoder_state)
        decoder_input = tf.argmax(decoder_output, -1)
        word_idx = decoder_input.numpy()[0][0]
        # if we've predicted 0 (which is reserved, usually this will only happen
        # before the decoder is trained, just stop translating and return
        # what we have)
        if word_idx == 0: 
          out_words.append('<end>')
        else:
          out_words.append(target_tokenizer.index_word[word_idx])

        if out_words[-1] == '<end>' or len(out_words) >= 20:
          break

    translation = ' '.join(out_words)    
    return source_sentences[idx], target_sentences[idx], translation
  
  
  @tf.function
  def train_step(self, source_seq, target_seq, target_labels, initial_state):
    with tf.GradientTape() as tape:
      encoder_output, encoder_state = self.encoder(source_seq, initial_state)
      logits, decoder_state = self.decoder(target_seq, encoder_state)
      loss = calc_loss(target_labels, logits)
    variables = self.encoder.trainable_variables + self.decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    self.optimizer.apply_gradients(zip(gradients, variables))
    return loss
  
  def train(self, EPOCHS=200):
    batch_size = self.batch_size
    dataset = self.t.dataset
    for epoch in range(EPOCHS):
      start = time.time()
      en_initial_states = self.encoder.init_state(batch_size)
      for batch, (source_seq, target_seq, target_labels) in enumerate(dataset):
        loss = self.train_step(source_seq, target_seq, target_labels, en_initial_states)
        elapsed = time.time() - start
      
      if epoch % 10 == 0:
        print("Epoch #%d, Loss %.4f, Time %.2f sec" % (epoch, loss, elapsed))
        input_sent, target_sent, translation = self.translate()
        print("Input: %s\nTarget: %s\nTranslation: %s\n" % (input_sent, target_sent, translation))
    

In [0]:
#trans = Trans(source_sentences, target_sentences)

In [215]:
# before trainings
#trans.translate()
#trans.translate_s('hello world ok ')
##print(trans.t.source_data.shape[1])

'<start>tropece sere labio mordio leseando chupa deje <end>'

In [0]:
#trans2 = Trans(target_sentences, source_sentences)


In [217]:
#trans2.translate_s(trans.translate_s('hello world ok '))

'<start>danced more all slow book deaf me snowed excited excited annoying ocd buy kissed loss resigned lion easter ? some'

**Translate English into Spanish**

In [0]:
trans = Trans(source_sentences, target_sentences)

In [219]:
trans.train(EPOCHS=300)

Epoch #0, Loss 1.7975, Time 4.26 sec
Input: <start> Be thorough . <end>
Target: <start> Se exhaustivo . <end>
Translation: ¿ . <end>

Epoch #10, Loss 0.9446, Time 1.43 sec
Input: <start> Bless you . <end>
Target: <start> Jesus . <end>
Translation: estamos . <end>

Epoch #20, Loss 0.6840, Time 1.42 sec
Input: <start> Are you okay ? <end>
Target: <start> ¿ Estas bien ? <end>
Translation: ¿ quien va ? <end>

Epoch #30, Loss 0.5747, Time 1.41 sec
Input: <start> I like them . <end>
Target: <start> Me gustan . <end>
Translation: me gusta el sushi . <end>

Epoch #40, Loss 0.5060, Time 1.42 sec
Input: <start> Now move on . <end>
Target: <start> Ahora prosigue . <end>
Translation: ven a echarme una mano . <end>

Epoch #50, Loss 0.4450, Time 1.42 sec
Input: <start> I m curious . <end>
Target: <start> Tengo curiosidad . <end>
Translation: tengo curiosidad . <end>

Epoch #60, Loss 0.3929, Time 1.44 sec
Input: <start> I m full . <end>
Target: <start> Estoy llena . <end>
Translation: estoy tan cansa

**BLEU score**

In [0]:
def eval(trans):
  references, hypotheses = [], []
  source_sentences = trans.t.source_sentences
  for i in range(len(source_sentences)):
    input_sent, target_sent, translation = trans.translate(i)
    references.append(target_sent)
    hypotheses.append('<start>' + translation)
  results = sacrebleu.raw_corpus_bleu(hypotheses, [references])
  print(results)

In [221]:
eval(trans)

BLEU(score=34.66402798501484, counts=[9246, 5958, 2959, 1071], totals=[13623, 10623, 7623, 4623], precisions=[67.87051310284079, 56.08585145439142, 38.81673881673882, 23.16677482154445], bp=0.8058872011091621, sys_len=13623, ref_len=16563)


#Part 2

**Translate Spanish into Engish**

In [222]:
trans2 = Trans(target_sentences, source_sentences)
trans2.train(EPOCHS=300)

Epoch #0, Loss 2.3342, Time 4.02 sec
Input: <start> Llovio . <end>
Target: <start> It rained . <end>
Translation: i i . <end>

Epoch #10, Loss 0.8774, Time 1.47 sec
Input: <start> Tiro la toalla . <end>
Target: <start> He gave up . <end>
Translation: i can t ? <end>

Epoch #20, Loss 0.7008, Time 1.46 sec
Input: <start> ¿ Estoy equivocada ? <end>
Target: <start> Am I wrong ? <end>
Translation: i m not fat . <end>

Epoch #30, Loss 0.6190, Time 1.66 sec
Input: <start> Soy escritor . <end>
Target: <start> I m a writer . <end>
Translation: i m a doctor . <end>

Epoch #40, Loss 0.4996, Time 1.45 sec
Input: <start> Tom es nuevo . <end>
Target: <start> Tom is new . <end>
Translation: tom s crazy . <end>

Epoch #50, Loss 0.3881, Time 1.48 sec
Input: <start> Estas loco ! <end>
Target: <start> You re nuts ! <end>
Translation: you re nuts ! <end>

Epoch #60, Loss 0.3106, Time 1.45 sec
Input: <start> ¿ Ha acabado Tom ? <end>
Target: <start> Is Tom done ? <end>
Translation: did tom come ? <end>

Epo

In [223]:
eval(trans2)

BLEU(score=46.29991332860898, counts=[10759, 7597, 4599, 1916], totals=[14200, 11200, 8200, 5200], precisions=[75.76760563380282, 67.83035714285714, 56.08536585365854, 36.84615384615385], bp=0.8110410418138186, sys_len=14200, ref_len=17174)


#Part 3

Back-translate

In [0]:
def back_translate(trans, trans2):
  references, hypotheses = [], []
  source_sentences = trans.t.source_sentences
  for s in source_sentences[:1000]:
    references.append(s)
    hypotheses.append(trans2.translate_s(trans.translate_s(s)))
  return references, hypotheses

references, hypotheses = back_translate(trans, trans2)
    

In [228]:
list(zip(references, hypotheses))[100:200]

[('<start> Go away ! <end>', '<start>keep it up ! <end>'),
 ('<start> Go away ! <end>', '<start>keep it up ! <end>'),
 ('<start> Go away ! <end>', '<start>keep it up ! <end>'),
 ('<start> Go away ! <end>', '<start>keep it up ! <end>'),
 ('<start> Go away ! <end>', '<start>keep it up ! <end>'),
 ('<start> Go away . <end>', '<start>keep it up ! <end>'),
 ('<start> Go away . <end>', '<start>keep it up ! <end>'),
 ('<start> Go away . <end>', '<start>keep it up ! <end>'),
 ('<start> Go away . <end>', '<start>keep it up ! <end>'),
 ('<start> Go away . <end>', '<start>keep it up ! <end>'),
 ('<start> Go away . <end>', '<start>keep it up ! <end>'),
 ('<start> Go away . <end>', '<start>keep it up ! <end>'),
 ('<start> Go home . <end>', '<start>stay close . <end>'),
 ('<start> Go slow . <end>', '<start>look help . <end>'),
 ('<start> Goodbye ! <end>', '<start>do come in ! <end>'),
 ('<start> Goodbye ! <end>', '<start>do come in ! <end>'),
 ('<start> Goodbye ! <end>', '<start>do come in ! <end>')

In [229]:
results = sacrebleu.raw_corpus_bleu(hypotheses, [references])
print('back-translate', results)

back-translate BLEU(score=6.501488223660218, counts=[2018, 943, 83, 14], totals=[4526, 3526, 2526, 1526], precisions=[44.5868316394167, 26.74418604651163, 3.285827395091053, 0.9174311926605505], bp=0.8396517914546833, sys_len=4526, ref_len=5317)


In [51]:
t1 = Translator(source_sentences, target_sentences)
print(t1.source_data[1000])
print(t1.target_data[1000])

print(t1.target_labels[200])
print(t1.source_vocab_size)
print(t1.target_vocab_size)

t1.decode(t1.source_data[1000])

ex_sentence = tf.expand_dims(t1.source_data[1000], axis=0)
ex_translation = tf.expand_dims(t1.target_data[1000], axis=0)
ex_labels = tf.expand_dims(t1.target_labels[1000], axis=0)
print(ex_sentence)
print(ex_translation)
print(ex_labels)

encoder = Encoder(t1)
hidden_state = encoder.init_state(1)
print(hidden_state.shape)
output, hidden_state = encoder(ex_sentence, hidden_state)
print(output.shape)

decoder = Decoder(t1)
decoder_output, decoder_state = decoder(ex_labels, hidden_state)
print(decoder_output.shape)

print("Loss", calc_loss(ex_labels, decoder_output))

[  1  13  12  17 432   3   2   0]
[  1  11   9 505   3   2   0   0   0   0   0]
[810.   3.   2.   0.   0.   0.   0.   0.   0.   0.   0.]
918
1909
1 -> <start>
13 -> he
12 -> is
17 -> a
432 -> dj
3 -> .
2 -> <end>
tf.Tensor([[  1  13  12  17 432   3   2   0]], shape=(1, 8), dtype=int32)
tf.Tensor([[  1  11   9 505   3   2   0   0   0   0   0]], shape=(1, 11), dtype=int32)
tf.Tensor([[ 11.   9. 505.   3.   2.   0.   0.   0.   0.   0.   0.]], shape=(1, 11), dtype=float64)
(1, 64)
(1, 8, 64)
(1, 11, 1909)
Loss tf.Tensor(3.4339483, shape=(), dtype=float32)


In [142]:
t = trans.t
(ssource, ttarget)=s = sentences[1000]
print(s)
ss_data = t.source_tokenizer.texts_to_sequences(ssource)
ss_data = tf.keras.preprocessing.sequence.pad_sequences(ss_data, padding='post')
print(len(ss_data))
een = trans.encoder
ddn = trans.decoder

input_sent = ss_data
input_send = tf.expand_dims(input_sent, axis=0)
print(input_send)
print(trans.t.source_data[1000])
print(trans.t.source_sentences[1000])
print(ssource)

('<start> He is a DJ . <end>', '<start> El es DJ . <end>')
26
tf.Tensor(
[[[  0]
  [  8]
  [ 21]
  [ 17]
  [  0]
  [ 21]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  4]
  [  8]
  [  0]
  [ 17]
  [  0]
  [354]
  [  0]
  [  0]
  [  3]
  [  0]
  [  0]
  [  0]
  [  0]
  [354]
  [  0]]], shape=(1, 26, 1), dtype=int32)
[  1  13  12  17 432   3   2   0]
<start> He is a DJ . <end>
<start> He is a DJ . <end>


In [197]:
print(ssource)
print(trans.t.source_sentences[1000])
print(trans.t.source_data[1000])

ssdata = trans.t.source_tokenizer.texts_to_sequences([ssource])
ssdata = tf.keras.preprocessing.sequence.pad_sequences(ssdata, maxlen=8, padding='post')
print(ssdata[0])

np.array(ssdata)
np.expand_dims(ssdata, axis=0)

<start> He is a DJ . <end>
<start> He is a DJ . <end>
[  1  13  12  17 432   3   2   0]
[  1  13  12  17 432   3   2   0]


array([[[  1,  13,  12,  17, 432,   3,   2,   0]]], dtype=int32)

In [148]:
print(trans.t.source_data.shape)
print(trans.t.target_data.shape)

(3000, 8)
(3000, 11)


In [135]:
help(trans.t.source_tokenizer)

Help on Tokenizer in module keras_preprocessing.text object:

class Tokenizer(builtins.object)
 |  Text tokenization utility class.
 |  
 |  This class allows to vectorize a text corpus, by turning each
 |  text into either a sequence of integers (each integer being the index
 |  of a token in a dictionary) or into a vector where the coefficient
 |  for each token could be binary, based on word count, based on tf-idf...
 |  
 |  # Arguments
 |      num_words: the maximum number of words to keep, based
 |          on word frequency. Only the most common `num_words-1` words will
 |          be kept.
 |      filters: a string where each element is a character that will be
 |          filtered from the texts. The default is all punctuation, plus
 |          tabs and line breaks, minus the `'` character.
 |      lower: boolean. Whether to convert the texts to lowercase.
 |      split: str. Separator for word splitting.
 |      char_level: if True, every character will be treated as a token.